In [11]:
# import dependencies
import pandas as pd
import numpy as np
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

warnings.filterwarnings("ignore")

### Pre-processing

In [12]:
df_106 = pd.read_csv("../Resources/106justice.csv")
df_106.tail()

,justice,term,direction,precedentAlteration,issueArea,lawType,caseOriginState
2731,106,2017,2,0,1,2,55
2732,106,2017,1,0,9,9,0
2733,106,2017,2,0,2,2,0
2734,106,2017,2,0,2,6,0
2735,106,2017,1,0,1,6,0


In [13]:
target_106 = df_106["direction"]
target_names = ["conservative", "liberal"]

In [14]:
data_106 = df_106.drop(["direction", "justice"], axis=1)
# feature_names = data_106.columns
data_106.head()

,term,precedentAlteration,issueArea,lawType,caseOriginState
0,1987,0,9,4,0
1,1987,0,10,1,5
2,1987,0,9,3,0
3,1987,0,7,6,0
4,1987,0,4,2,42


In [15]:
# one_hot_issueArea = to_categorical(data_106["issueArea"])
# one_hot_issueArea

In [16]:
term_df = pd.get_dummies(data_106["term"], prefix="term")
precAlt_df = pd.get_dummies(data_106["precedentAlteration"], prefix="precedentAlteration")
issueArea_df = pd.get_dummies(data_106["issueArea"], prefix="issueArea")
lawType_df = pd.get_dummies(data_106["lawType"], prefix="lawType")
caseOriginState_df = pd.get_dummies(data_106["caseOriginState"], prefix="caseOriginState")

In [17]:
oneHot_features = pd.concat([term_df, precAlt_df], axis=1)
oneHot_features = pd.concat([oneHot_features, issueArea_df], axis=1)
oneHot_features = pd.concat([oneHot_features, lawType_df], axis=1)
oneHot_features = pd.concat([oneHot_features, caseOriginState_df], axis=1)
oneHot_features.head()

,term_1987,term_1988,term_1989,term_1990,term_1991,term_1992,term_1993,term_1994,term_1995,term_1996,...,caseOriginState_49,caseOriginState_50,caseOriginState_51,caseOriginState_52,caseOriginState_53,caseOriginState_55,caseOriginState_56,caseOriginState_57,caseOriginState_58,caseOriginState_59
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
X = oneHot_features
feature_names = oneHot_features.columns
y = target_106
feature_names.ravel()

array(['term_1987', 'term_1988', 'term_1989', 'term_1990', 'term_1991',
       'term_1992', 'term_1993', 'term_1994', 'term_1995', 'term_1996',
       'term_1997', 'term_1998', 'term_1999', 'term_2000', 'term_2001',
       'term_2002', 'term_2003', 'term_2004', 'term_2005', 'term_2006',
       'term_2007', 'term_2008', 'term_2009', 'term_2010', 'term_2011',
       'term_2012', 'term_2013', 'term_2014', 'term_2015', 'term_2016',
       'term_2017', 'precedentAlteration_0', 'precedentAlteration_1',
       'issueArea_1', 'issueArea_2', 'issueArea_3', 'issueArea_4',
       'issueArea_5', 'issueArea_6', 'issueArea_7', 'issueArea_8',
       'issueArea_9', 'issueArea_10', 'issueArea_12', 'issueArea_13',
       'issueArea_14', 'lawType_0', 'lawType_1', 'lawType_2', 'lawType_3',
       'lawType_4', 'lawType_5', 'lawType_6', 'lawType_8', 'lawType_9',
       'caseOriginState_0', 'caseOriginState_1', 'caseOriginState_2',
       'caseOriginState_4', 'caseOriginState_5', 'caseOriginState_6',
       

### Evaluating different classifiers

In [19]:
clf_lr = LogisticRegression()
clf_rf = RandomForestClassifier()
clf_nb = GaussianNB()
clf_svm = SVC(kernel='linear', probability=True)

print('5-fold cross validation:\n')

labels = ["Logistic Regression", "Random Forest", "Naive Bayes", "Linear SVM"]

for clf, label in zip([clf_lr, clf_rf, clf_nb, clf_svm], labels):
    scores = cross_val_score(clf, X, y, cv=5, 
                             scoring='f1', n_jobs=-1)
    print("F1: %0.3f [%s]" % (scores.mean(), label))


5-fold cross validation:

F1: 0.616 [Logistic Regression]
F1: 0.596 [Random Forest]
F1: 0.111 [Naive Bayes]
F1: 0.654 [Linear SVM]


In [20]:
# using all but naive bayes (since score was low) for voting classifier estimators
v_clf_hard = VotingClassifier(estimators = [(labels[0], clf_lr),
                                            (labels[1], clf_rf),
                                            (labels[3], clf_svm)],
                                            voting = 'hard')

In [21]:
v_clf_soft = VotingClassifier(estimators = [(labels[0], clf_lr),
                                            (labels[1], clf_rf),
                                            (labels[3], clf_svm)],
                                            voting = 'soft')

In [22]:
new_labels = ["Logistic Regression", "Random Forest", "Linear SVM", 
              "Voting_Classifier_Hard", "Voting_Classifier_Soft"]

for (clf, label) in zip([clf_lr, clf_rf, clf_svm, v_clf_hard, v_clf_soft], new_labels):
    scores = cross_val_score(clf, X, y,cv=5,
                             scoring='f1', n_jobs=-1)
    print("F1: %0.3f [%s]" % (scores.mean(), label))

F1: 0.616 [Logistic Regression]
F1: 0.615 [Random Forest]
F1: 0.654 [Linear SVM]
F1: 0.628 [Voting_Classifier_Hard]
F1: 0.644 [Voting_Classifier_Soft]


### Training the SVM model

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
model = SVC(kernel='linear')
model.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [25]:
# Model Accuracy
print('Train Acc: %.3f' % model.score(X_train, y_train))
print('Test Acc: %.3f' % model.score(X_test, y_test))

Train Acc: 0.607
Test Acc: 0.628


In [26]:
predictions = model.predict(X_test)
print(classification_report(y_test, predictions,
                            target_names=target_names))

              precision    recall  f1-score   support

conservative       0.63      0.90      0.75       330
     liberal       0.59      0.21      0.31       218

   micro avg       0.63      0.63      0.63       548
   macro avg       0.61      0.56      0.53       548
weighted avg       0.62      0.63      0.57       548



In [30]:
print(confusion_matrix(predictions, y_test))
predictions[:15], y_test[:15].ravel()

[[298 172]
 [ 32  46]]


(array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1], dtype=int64),
 array([1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 1, 2], dtype=int64))

### Save Model and Confirm Load

In [49]:
#Save Model
#https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/
import pickle

filename = '106_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [50]:
#Load Model from Disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.6277372262773723
